# Week 4 Checkin - Logistic Regression

In [15]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt


In [18]:
logistic_data_orig = pd.read_excel('../hannah/clean_data.xlsx')
logistic_data_orig

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88727,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
88728,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
88729,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
88730,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [19]:
logistic_data_orig.sample(7)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
12064,13013,2VAMRYzi5TM5GChU6pTtIE,Mark Farina,Air Farina,Dream Machine - Downtempo Mix,45,368200,False,0.938,0.424,...,-10.299,1,0.0590,0.146000,0.002090,0.2620,0.652,110.783,4,chicago-house
48202,55568,4xWBa63JWfBQ9P7yReHtps,Pritam;Arijit Singh,Ae Dil Hai Mushkil (Original Motion Picture So...,Channa Mereya,49,289333,False,0.476,0.788,...,-4.821,0,0.0446,0.237000,0.000025,0.1060,0.746,90.066,4,indian
49267,58102,6UIo6YbaXihIZ72MWUpcGb,Marilyn Manson,Heaven Upside Down,KILL4ME,59,239240,True,0.635,0.688,...,-6.136,0,0.0346,0.002300,0.000011,0.5070,0.389,107.989,4,industrial
20567,22003,4Ku3OOZWR4tpKNCVFXILY8,Suicide Silence,The Black Crown,You Only Live Once,60,192560,False,0.613,0.953,...,-4.936,1,0.1530,0.000180,0.061400,0.1160,0.213,119.976,4,death-metal
52482,61703,0PkWP3XfbPI9WRMYbYAnKD,Kyary Pamyu Pamyu,ゆめのはじまりんりん,スローモ,19,213977,False,0.499,0.937,...,-4.402,0,0.0400,0.018100,0.001180,0.0583,0.471,139.986,4,j-idol
42898,48997,7lgb2qegIOmODRmVJBmDo0,Seringai,Taring,"Fett, Sang Pemburu",30,255400,False,0.553,0.911,...,-4.771,1,0.0380,0.000017,0.008370,0.1300,0.276,109.950,4,hardcore
59822,72403,7fQox8Ere1HDm0jhSnsFHu,Bring Me The Horizon;YUNGBLUD,Obey (with YUNGBLUD),Obey (with YUNGBLUD),55,220538,True,0.462,0.990,...,-3.175,0,0.2110,0.000103,0.000000,0.3740,0.204,137.061,4,metalcore



---


## Fitting logistic regression to the full training dataset


Next, compute a logistic regression fit for the entire training dataset:

In [20]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Separate the features and target variable
X = logistic_data_orig[["instrumentalness", "speechiness", "energy", "valence", "danceability", "acousticness"]]

y = (logistic_data_orig['popularity'] > 30).astype(int)  # Convert to binary: 1 if popularity > 33, else 0

# Apply LabelEncoder to each categorical column in X
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Split data into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the logistic regression model with binary target
lr_all = LogisticRegression(solver='liblinear')
lr_all.fit(X_train, y_train)

# View intercept and coefficients
print("Intercept:", lr_all.intercept_)
print("Coefficients:", lr_all.coef_)




Intercept: [0.17933063]
Coefficients: [[-0.96508481 -2.19179382 -0.10068603 -0.76492574  1.34050641 -0.04720315]]


In [21]:
# Randomly select 50 sample indices from X_val
sample_indices = np.random.choice(X_val.index, size=50, replace=False)

# Select samples from X_val and y_val using the random indices
X_val_sample = X_val.loc[sample_indices, :]
y_val_sample = y_val.loc[sample_indices]

# Predict probabilities and classes for the sample
y_val_pred_probs = lr_all.predict_proba(X_val_sample)[:, 1]  # Probability for the positive class
y_val_pred_classes = lr_all.predict(X_val_sample)

# Create a DataFrame to display actual vs. predicted
evaluation_df = pd.DataFrame({
    'Is popular': y_val_sample,
    'Predicted Probability': y_val_pred_probs,
    'Predicted Popularity': y_val_pred_classes
})

# Display the evaluation DataFrame
print(evaluation_df)


       Is popular  Predicted Probability  Predicted Popularity
36004           1               0.617382                     1
65496           1               0.600216                     1
43797           1               0.366267                     0
21099           0               0.463447                     0
87329           1               0.634489                     1
8332            1               0.537260                     1
42775           1               0.587834                     1
76512           1               0.538421                     1
44532           0               0.518543                     1
15940           1               0.691519                     1
67053           0               0.620270                     1
10454           0               0.341174                     0
73957           0               0.561523                     1
60042           0               0.506154                     1
31804           1               0.561707               

In [22]:
y_val_pred = lr_all.predict(X_val)
conf_lr = metrics.confusion_matrix(y_true=y_val, y_pred=y_val_pred)
conf_lr

array([[2943, 5169],
       [2195, 7440]])

Prediction accuracy:

In [23]:
# Logistic regression
print((conf_lr[0, 0] + conf_lr[1, 1]) / conf_lr.sum())

0.5850566292894573


In [24]:
# true positive rate
tpr= (conf_lr[1,1])/conf_lr[1,:].sum()
# true negative rate
tnr= (conf_lr[0,0])/conf_lr[0,:].sum()

print("True Positive Rate:",tpr)
print("True Negative Rate:",tnr)

True Positive Rate: 0.772184743124027
True Negative Rate: 0.36279585798816566


***Predicted probability densities***


In [25]:
px.histogram(evaluation_df, x='Predicted Probability', color='Is popular',
             nbins=20, opacity=0.5,
             barmode='overlay')



***ROC Curve***

In [26]:
lr_fpr_sample, lr_tpr_sample, lr_thresholds_sample = metrics.roc_curve(evaluation_df['Is popular'], evaluation_df['Predicted Probability'])

lr_thresholds_sample


array([       inf, 0.69151883, 0.64459139, 0.6334734 , 0.62027045,
       0.61738187, 0.6051238 , 0.60021594, 0.59539167, 0.57357692,
       0.57091076, 0.56679469, 0.56411851, 0.56170721, 0.53983513,
       0.53725993, 0.52915411, 0.52895176, 0.50615406, 0.48933156,
       0.40265944, 0.40259042, 0.39983226, 0.3662665 , 0.34117425,
       0.32850964, 0.29461605])

In [27]:
roc_lr_sample = pd.DataFrame({
    'False Positive Rate': lr_fpr_sample,
    'True Positive Rate': lr_tpr_sample,
    'Model': 'Logistic Regression'
}, index=lr_thresholds_sample)


roc_sample_df = pd.concat([roc_lr_sample])


px.line(roc_sample_df, y='True Positive Rate', x='False Positive Rate',
        color='Model',
        width=700, height=500
)

In [28]:
# Logistic regression
lr_auc_sample = metrics.roc_auc_score(evaluation_df['Is popular'], evaluation_df['Predicted Probability'])
print('Logistic regression AUC:', lr_auc_sample.round(3))

Logistic regression AUC: 0.523


***USING CV FOR MODEL PERFORMANCE***

In [29]:
from sklearn.model_selection import cross_val_score


# Separate the features and target variable
X = logistic_data_orig[["instrumentalness", "speechiness", "energy", "valence", "danceability", "acousticness"]]

y = (logistic_data_orig['popularity'] > 30).astype(int)  # Convert to binary: 1 if popularity > 33, else 0

# This does stratified Kfolds for us...
cross_val_score(lr_all, X, y, cv=5, scoring='roc_auc')

array([0.61194458, 0.61632333, 0.62607415, 0.53816304, 0.48793798])

In [30]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

# Use the shuffle and random state if want data shuffled before splitting
#skfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skfolds = StratifiedKFold(n_splits=5)
i = 1
for train_index, test_index in skfolds.split(X, y):
    clone_lr = clone(lr_all)
    X_train_folds = X.iloc[train_index]
    y_train_folds = y.iloc[train_index]
    X_test_fold = X.iloc[test_index]
    print(test_index)
    clone_lr.fit(X_train_folds, y_train_folds)
    y_pred = clone_lr.predict(X_test_fold)

    auc_sample = metrics.roc_auc_score(y.iloc[test_index], y_pred)
    print('Fold: ', i)
    print('AUC: ', auc_sample)
    print('Accuracy: ', metrics.accuracy_score(y.iloc[test_index], y_pred))
    i += 1

[    0     1     2 ... 19686 19687 19689]
Fold:  1
AUC:  0.5793129726060839
Accuracy:  0.5916492928382262
[17120 17121 17122 ... 36795 36796 36797]
Fold:  2
AUC:  0.583511244232712
Accuracy:  0.599199864765876
[34994 34995 34996 ... 57555 57556 57557]
Fold:  3
AUC:  0.5884538204495349
Accuracy:  0.6043051955370224
[50360 50361 50362 ... 73050 73056 73057]
Fold:  4
AUC:  0.5262817018206203
Accuracy:  0.543615462639468
[67736 67737 67738 ... 88729 88730 88731]
Fold:  5
AUC:  0.4887287537481254
Accuracy:  0.5067620872309253
